In [ ]:
#SENTIMENT ANALYSIS CELLS
#MERGED DATASET WITH KEYWORDS FEATURES WILL BE USED (HERE IS A PLACEHOLDER)
#Libraries: feel-it, google-trans, VADERSentiment, Textblob

#FEEL-IT
import pandas as pd

df_books = "PLACEHOLDER"  #MERGED DATASET IN CSV FORMAT
list_contents = df_books['content2'].to_list()

from feel_it import EmotionClassifier, SentimentClassifier

emotion_classifier = EmotionClassifier()
list_emotions = emotion_classifier.predict(list_contents)

sentiment_classifier = SentimentClassifier()
list_sentiments = sentiment_classifier.predict(list_contents)

df_books['sentiment'] = list_sentiments
df_books['emotion'] = list_emotions

#GOOGLE-TRANS OBJECT INSTANTIATION
from googletrans import Translator
translator = Translator()

list_contents = df_books['content2'].to_list()

#ENGLISH DESCRIPTIONS (TIME IS USED TO AVOID PROBLEMS WITH GOOGLE API REQUESTS CAP)
import time
for content in list_contents:
  content = content.replace(",", " ").replace("<", " ").replace(">", " ").replace(".", " ").replace("  ", " ").replace("&", " ")
  if not content[-1].isalpha():
    content = content[0:len(content) - 1]
  print(content)
  time.sleep(0.3)
  if len(str(content)) <= 5000:
    translation = translator.translate(str(content), src="it", dest="en")
  else:
    #print("TRIGGERED")
    translaction = translator.translate(str(content[0:4999]), src="it", dest="en")
  list_content_en.append(translation.text)

df_books["content_english"] = list_content_en

#TEXTBLOB (SPACY SUB-MODULE)
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import pandas as pd

list_content_en = df_books['content_english'].to_list()

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')
list_polarities_textblob = []
for content in list_content_en:
  doc = nlp(content)
  if doc._.blob.polarity >= 0:
    list_polarities_textblob.append("positive")
  else:
    list_polarities_textblob.append("negative")

df_books["textblob_english_sentiment"] = list_polarities_textblob

#VADER
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

list_contents = df_books['content_english'].to_list()
list_score = []

for desc in list_contents:
    if analyzer.polarity_scores(desc)['compound'] <= 0:
        list_score.append('negative')
    else:
        list_score.append('positive')

df_books['vader_score'] = list_score

#MAJORITY VOTING
def prova(x):
  counter_positive = 0
  counter_negative = 0

  if x['vader_score'] == 'positive':
    counter_positive += 1
  else:
    counter_negative += 1
  
  if x['sentiment'] == 'positive':
    counter_positive += 1
  else:
    counter_negative += 1

  if x['textblob_english_sentiment'] == 'positive':
    counter_positive += 1
  else:
    counter_negative += 1
  
  return 'positive' if counter_positive > counter_negative else 'negative'

df_books['majority'] = df_books.apply(prova, axis=1)
df_books.to_csv('2_books_with_majority.csv') #WHERE THE FILE IS SAVED